In [1]:
# Install necessary libraries
!pip install transformers datasets wandb torch psutil scikit-learn sacremoses

# Import required modules
import wandb
import psutil
import pandas as pd
import torch
import sacremoses
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
)
from sklearn.model_selection import train_test_split

from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 13.0 MB/s eta 0:00:00


In [2]:
# Load dataset
dataset_path = "/kaggle/input/ban-pl-1/BAN-PL.csv"  # Update this path
data = pd.read_csv(dataset_path, encoding="utf-8")

# Preview the dataset
print(data.head())

     id                                               Text  Class
0  2200  \n\n\n            Polska wtedy oficjalnie powi...      0
1  2201  \n  Gigantyczna różnica\n\n{USERNAME}: biorac ...      0
2  2202  \n\n            {USERNAME}: Moj kumpel budowla...      0
3  2203         kura, rzodkiewka za 3pln to nie jest tanio      0
4  2204  {USERNAME}: większość nie idzie w marszu za PO...      0


In [3]:
# Load HerBERT tokenizer
tokenizer = AutoTokenizer.from_pretrained("allegro/herbert-base-cased")

# Tokenize text
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=128)

# Split dataset into training and validation sets
texts = data["Text"].tolist()
labels = data["Class"].tolist()

train_texts, val_texts, train_labels, val_labels = train_test_split(
    texts, labels, test_size=0.2, random_state=42
)

tokenizer_config.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/472 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/907k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/556k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/129 [00:00<?, ?B/s]

In [4]:
# Tokenize splits
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128, return_tensors="pt")
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=128, return_tensors="pt")

# Define PyTorch dataset class
class BANPLDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

# Create PyTorch datasets
train_dataset = BANPLDataset(train_encodings, train_labels)
val_dataset = BANPLDataset(val_encodings, val_labels)

In [5]:
# Load the pre-trained HerBERT model
model = AutoModelForSequenceClassification.from_pretrained("allegro/herbert-base-cased", num_labels=2)

# Define training arguments
training_args = TrainingArguments(
    output_dir="/kaggle/working/results",          # output directory
    evaluation_strategy="epoch",         # evaluate at the end of each epoch
    save_strategy="epoch",
    logging_dir="/kaggle/working/results/logs",           # directory for storing logs
    per_device_train_batch_size=32,      # smaller batch size for more granular updates
    per_device_eval_batch_size=32,       # smaller batch size for evaluation
    gradient_accumulation_steps=2,       # accumulate gradients over multiple steps
    learning_rate=1e-5,                  # lower learning rate
    num_train_epochs=5,                  # fewer epochs
    weight_decay=0.01,                   # regularization
    fp16=True,                           # mixed precision training
    load_best_model_at_end=True,         # load the best model when finished
    metric_for_best_model="accuracy",    # use accuracy for determining the best model
    greater_is_better=True,              # higher is better
    report_to="wandb",                   # log to W&B
    remove_unused_columns=False,         # ensure all columns are used
)

pytorch_model.bin:   0%|          | 0.00/654M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at allegro/herbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [6]:
# Initialize W&B
wandb.login(key=user_secrets.get_secret("wandb_key"))
wandb.init(
    project="herbert-hate-detector",  #W&B Project name
    entity="m-baloniakk",            #W&B entity
    name="fine-tune-ban-run-2",       #W&B Run Name
    config=training_args.to_dict(),
    settings=wandb.Settings(start_method="fork", _disable_stats=True)  # Fixes some runtime issues with multiprocessing
)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: m-baloniak (m-baloniakk). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Tracking run with wandb version 0.19.1
wandb: Run data is saved locally in /kaggle/working/wandb/run-20250122_230215-vzhhlluz
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run fine-tune-ban-run-2
wandb: ⭐️ View project at https://wandb.ai/m-baloniakk/herbert-hate-detector
wandb: 🚀 View run at https://wandb.ai/m-baloniakk/herbert-hate-detector/runs/vzhhlluz


In [7]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

# Define the metrics function
def compute_metrics(p):
    preds, labels = p
    preds = preds.argmax(axis=-1)  # Get the predicted class labels (max probability)

    accuracy = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average='weighted')
    precision = precision_score(labels, preds, average='weighted')
    recall = recall_score(labels, preds, average='weighted')
    
    return {
        'accuracy': accuracy,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [8]:
trainer = Trainer(
    model=model,                         # your model
    args=training_args,                  # training arguments
    train_dataset=train_dataset,         # your training dataset
    eval_dataset=val_dataset,           # your eval dataset
    tokenizer=tokenizer,                 # tokenizer
    compute_metrics=compute_metrics,     # metrics function (e.g., accuracy)
)

# Train the model
trainer.train()

# Evaluate the model
results = trainer.evaluate()

# Finish W&B run
wandb.finish()

<ipython-input-8-8ae809ab0ab9>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
<ipython-input-4-bfd04aae6458>:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,0.268217,0.890000,0.890000,0.890006,0.890000
2,No log,0.236345,0.910625,0.910623,0.910643,0.910625
3,No log,0.229794,0.913750,0.913750,0.913759,0.913750
4,0.275400,0.229170,0.914583,0.914583,0.914587,0.914583
5,0.275400,0.229293,0.912917,0.912906,0.913050,0.912917


<ipython-input-4-bfd04aae6458>:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
<ipython-input-4-bfd04aae6458>:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were sc

wandb:                                                                                
wandb: 
wandb: Run history:
wandb:           eval/accuracy ▁▇████
wandb:                 eval/f1 ▁▇████
wandb:               eval/loss █▂▁▁▁▁
wandb:          eval/precision ▁▇████
wandb:             eval/recall ▁▇████
wandb:            eval/runtime ▁██▇▅▆
wandb: eval/samples_per_second █▁▁▂▄▃
wandb:   eval/steps_per_second █▁▁▂▄▃
wandb:             train/epoch ▁▃▅▅▆███
wandb:       train/global_step ▁▃▅▅▆███
wandb:         train/grad_norm ▁
wandb:     train/learning_rate ▁
wandb:              train/loss ▁
wandb: 
wandb: Run summary:
wandb:            eval/accuracy 0.91458
wandb:                  eval/f1 0.91458
wandb:                eval/loss 0.22917
wandb:           eval/precision 0.91459
wandb:              eval/recall 0.91458
wandb:             eval/runtime 21.7504
wandb:  eval/samples_per_second 220.686
wandb:    eval/steps_per_second 3.448
wandb:               total_flos 6314665328640000.0
wandb